In [1]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi 

Thu Aug 25 23:50:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary

In [4]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST dataset
# pre-processor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = torchvision.datasets.MNIST(root='data/',
                                           train=True, 
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/',
                                          train=False, 
                                          transform=transform)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False, num_workers=2)

In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.LSTM(28, 128, 1, batch_first=True)
        self.layer2 = nn.LSTM(128, 64, 1, batch_first=True)
        self.fc1 = nn.Linear(7*8*32, 480)
        self.fc2   = nn.Linear(480, 120)
        self.fc3   = nn.Linear(120, num_classes)

    def forward(self, x):
        h1 = torch.randn(1, x.size(0), 128).to(device)
        c1 = torch.randn(1, x.size(0), 128).to(device)
        out, (hn, cn) = self.layer1(x, (h1, c1))

        h2 = torch.randn(1, out.size(0), 64).to(device)
        c2 = torch.randn(1, out.size(0), 64).to(device)
        out, (hn, cn) = self.layer2(out, (h2, c2))

        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
summary(model, input_size=(batch_size, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
ConvNet                                  [100, 10]                 --
├─LSTM: 1-1                              [100, 28, 128]            80,896
├─LSTM: 1-2                              [100, 28, 64]             49,664
├─Linear: 1-3                            [100, 480]                860,640
├─Linear: 1-4                            [100, 120]                57,720
├─Linear: 1-5                            [100, 10]                 1,210
Total params: 1,050,130
Trainable params: 1,050,130
Non-trainable params: 0
Total mult-adds (M): 457.52
Input size (MB): 0.31
Forward/backward pass size (MB): 4.79
Params size (MB): 4.20
Estimated Total Size (MB): 9.30

In [8]:

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28, 28)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.4178
Epoch [1/5], Step [200/600], Loss: 0.2301
Epoch [1/5], Step [300/600], Loss: 0.1592
Epoch [1/5], Step [400/600], Loss: 0.3165
Epoch [1/5], Step [500/600], Loss: 0.0917
Epoch [1/5], Step [600/600], Loss: 0.0758
Epoch [2/5], Step [100/600], Loss: 0.1852
Epoch [2/5], Step [200/600], Loss: 0.0612
Epoch [2/5], Step [300/600], Loss: 0.0637
Epoch [2/5], Step [400/600], Loss: 0.1153
Epoch [2/5], Step [500/600], Loss: 0.0479
Epoch [2/5], Step [600/600], Loss: 0.1104
Epoch [3/5], Step [100/600], Loss: 0.0500
Epoch [3/5], Step [200/600], Loss: 0.0804
Epoch [3/5], Step [300/600], Loss: 0.0958
Epoch [3/5], Step [400/600], Loss: 0.0335
Epoch [3/5], Step [500/600], Loss: 0.0496
Epoch [3/5], Step [600/600], Loss: 0.1031
Epoch [4/5], Step [100/600], Loss: 0.0568
Epoch [4/5], Step [200/600], Loss: 0.0342
Epoch [4/5], Step [300/600], Loss: 0.0445
Epoch [4/5], Step [400/600], Loss: 0.0337
Epoch [4/5], Step [500/600], Loss: 0.0177
Epoch [4/5], Step [600/600], Loss:

In [9]:
# Test the model
import time


model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

start_time = time.time()
with torch.no_grad():
    correct = 0
    total = 0
    pred =[]
    y_true = []
    for images, labels in test_loader:
        images = images.reshape(-1, 28, 28)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        pred+=predicted.tolist()
        y_true+=labels.tolist()

#     print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.835646629333496 seconds ---


In [10]:
from sklearn.metrics import classification_report
print(classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      0.99      0.99      1135
           2       0.97      1.00      0.98      1032
           3       0.99      0.98      0.99      1010
           4       0.99      0.98      0.99       982
           5       0.99      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.99      0.98      0.98      1028
           8       0.99      0.98      0.99       974
           9       0.99      0.98      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [11]:
# import torch.autograd.profiler as profiler
# with profiler.profile(with_stack=True, profile_memory=True) as prof:
#     out = model(images)

# print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))